# How Time Works

In the graph there is a node labled `time` with no edges. At the beginning `currentTime=0`, and is autoincremented by 1 indefinitely. 

An Azure function `functions/time` runs all events and actions that require time to complete. That runs on an NCRONTAB job specified in the `function.json` doc. It can be adjusted to speed up or slow down time in the game. 

Example NCRONTAB configurations:
* `"0 */2 * * * *"`  Every two minutes
* `"0 */5 * * * *"`  Every five minutes


In [2]:
import sys
import pandas as pd
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f

import ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [3]:
time = {
    "label": "time",
    "objid":  f.uuid(n=13),
    "name": "time",
    "originTime":0,
    "currentTime":0
}
time

{'label': 'time',
 'objid': '1055528131079',
 'name': 'time',
 'originTime': 0,
 'currentTime': 0}

Time has no edges. It sits in it's own record. 

**Note** The cell below will reset time. This will cause disruptions in the larger system. 

In [4]:
# # res = db.run_query("g.V().hasLabel('time').drop()")
# res = db.run_query("g.addV('time').property('label','time').property('objid','7263327370874').property('name','time').property('originTime',0).property('currentTime',0).property('username','notebook').property('objtype','time')")
# res = db.run_query("g.V().hasLabel('time').valueMap()")
# time = [db.clean_node(n) for n in res][0]
# time

In [5]:
res = db.run_query("g.V().hasLabel('time').valueMap()")
time = [db.clean_node(n) for n in res][0]
time

{'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'currentTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}

In [6]:
query = '''
    g.V().haslabel("time").limit(1).values('currentTime')
    '''
res = db.run_query(query)

res


[0]

**TODO** Update this function to get the time and update the time in a single query. 

In [1]:
# query = '''
#     g.V().haslabel("time").limit(1).property("currentTime", union(values("currentTime"), constant(1)).sum())
#     '''
# res = db.run_query(query)

# res
# # g.V().haslabel("time").limit(1).values('currentTime')+1

Currently the process is mundane: get the value of `currentTime` and add one. Then place that property. 

In [13]:
currentTime = res[0]+1
updateRes = db.run_query(f"g.V().hasLabel('time').property('currentTime', {currentTime})")
updateRes

[{'id': '9f20308e-c85e-414f-b2ee-77316df94689',
  'label': 'time',
  'type': 'vertex',
  'properties': {'currentTime': [{'id': '84114321-4b70-4812-b555-f758761b5ed8',
     'value': 1}],
   'objid': [{'id': '9f20308e-c85e-414f-b2ee-77316df94689|objid',
     'value': '7263327370874'}],
   'name': [{'id': '3db56964-4c7a-4a9a-a7a3-2d186153e998', 'value': 'time'}],
   'originTime': [{'id': 'a8af8453-3962-49d9-ba22-447f0f48b6ca', 'value': 0}],
   'username': [{'id': 'e85d9ed1-5492-4524-b042-efbaa7064449',
     'value': 'notebook'}],
   'objtype': [{'id': '6e7e1ebe-b927-485c-973e-28a3ced14afc',
     'value': 'time'}]}}]

In [14]:
res = db.run_query("g.V().hasLabel('time').valueMap()")
time = [db.clean_node(n) for n in res][0]
time

{'currentTime': 1,
 'objid': '7263327370874',
 'name': 'time',
 'originTime': 0,
 'username': 'notebook',
 'objtype': 'time',
 'id': '7263327370874'}